## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-20-17-19-12 +0000,world,Japan’s ruling party projected to lose majorit...,https://www.washingtonpost.com/world/2025/07/2...
1,2025-07-20-16-06-54 +0000,world,Trump’s bullying of Brazil is backfiring,https://www.washingtonpost.com/world/2025/07/2...
2,2025-07-20-11-49-34 +0000,world,Typhoon Wipha hits mainland China after disrup...,https://www.washingtonpost.com/world/2025/07/2...
3,2025-07-20-09-00-00 +0000,world,U.S. citizen who works for Commerce Dept. ensn...,https://www.washingtonpost.com/national-securi...
4,2025-07-20-08-00-49 +0000,world,See how Nigeria’s economic crisis has changed ...,https://www.washingtonpost.com/world/interacti...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,village,2
0,japan,1
2,party,1
1,ruling,1
4,lose,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-07-20-06-00-38 +0000,world,Killings near a West Bank village show even Am...,https://www.washingtonpost.com/world/2025/07/2...,11
0,2025-07-20-17-19-12 +0000,world,Japan’s ruling party projected to lose majorit...,https://www.washingtonpost.com/world/2025/07/2...,9
6,2025-07-20-04-30-39 +0000,world,Tourist boat capsizes in Vietnam’s Ha Long Bay...,https://www.washingtonpost.com/world/2025/07/2...,9
2,2025-07-20-11-49-34 +0000,world,Typhoon Wipha hits mainland China after disrup...,https://www.washingtonpost.com/world/2025/07/2...,9
3,2025-07-20-09-00-00 +0000,world,U.S. citizen who works for Commerce Dept. ensn...,https://www.washingtonpost.com/national-securi...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,11,2025-07-20-06-00-38 +0000,world,Killings near a West Bank village show even Am...,https://www.washingtonpost.com/world/2025/07/2...
0,9,2025-07-20-17-19-12 +0000,world,Japan’s ruling party projected to lose majorit...,https://www.washingtonpost.com/world/2025/07/2...
2,9,2025-07-20-11-49-34 +0000,world,Typhoon Wipha hits mainland China after disrup...,https://www.washingtonpost.com/world/2025/07/2...
6,9,2025-07-20-04-30-39 +0000,world,Tourist boat capsizes in Vietnam’s Ha Long Bay...,https://www.washingtonpost.com/world/2025/07/2...
3,8,2025-07-20-09-00-00 +0000,world,U.S. citizen who works for Commerce Dept. ensn...,https://www.washingtonpost.com/national-securi...
7,7,2025-07-19-18-13-54 +0000,world,"As U.S. hails Syria ceasefire, Sweida resident...",https://www.washingtonpost.com/world/2025/07/1...
1,4,2025-07-20-16-06-54 +0000,world,Trump’s bullying of Brazil is backfiring,https://www.washingtonpost.com/world/2025/07/2...
4,4,2025-07-20-08-00-49 +0000,world,See how Nigeria’s economic crisis has changed ...,https://www.washingtonpost.com/world/interacti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
